In [ ]:
# Here's the Swagger UI documentation of an API service called Diction- a collection of inspiring quotes from famous authors.: https://api.fisenko.net/swagger-ui/
# Students are required to go through the documentation, and create a jupyter notebook that makes  REST API calls to this service to:
# Get the number of languages supported. Note that you'll need the language id for making other API calls.
# Randomly generate a quote, and store the retrieved information in a csv/ db table.
# Fetch the list of authors in each language and persist the results.
# For a specific author, get the corresponding author id from the previous result, and fetch the famous quotes by the same author. Persist the results.
# Provide a feature to like a quote. Use the Post API call for this.
#  Allow users to submit quotes. 
# Get the total number of quotes and authors in each language

In [1]:
import requests
import json
import pandas as pd

In [2]:
BASE_URL = "https://api.fisenko.net/v1"


def get_languages():
    response = requests.get(f"{BASE_URL}/languages")
    languages = response.json()
    return pd.DataFrame(languages)

languages_df = get_languages()
print(languages_df)


  code language
0   ru  Русский
1   en  English


In [3]:
def get_random_quote(language_id):
    response = requests.get(f"{BASE_URL}/quotes/{language_id}/random")
    quote_data = response.json()
    return quote_data

# Fetch a random quote for English
random_quote = get_random_quote("en")

quote_df = pd.DataFrame([random_quote])
quote_df.to_csv("random_quote.csv", index=False)
quote_df

,id,text,author
0,6153bbb59e8e5ae3eb2429a6,My faith helps me overcome such negative emoti...,"{'id': '6153b7d49e8e5ae3eb22fd1e', 'name': 'Da..."


In [4]:
def get_authors_by_language(language_id):
    response = requests.get(f"{BASE_URL}/authors/{language_id}")
    quote_data = response.json()
    return quote_data

authors_list = {}
for lang in languages_df['code']:
    authors_list[lang] = get_authors_by_language(lang)
authors_list

{'ru': [{'id': '6153b7d49e8e5ae3eb22fcbf', 'name': 'Aл-Алави'},
  {'id': '6153b7d49e8e5ae3eb22fd1a', 'name': 'Cтефан Цвейг'},
  {'id': '6153b7d49e8e5ae3eb22fe95', 'name': 'O. Бальзак'},
  {'id': '61f7f666a4755409f15f1dc9', 'name': 'Traf'},
  {'id': '6153b7d49e8e5ae3eb22ff5f', 'name': '«В погоне за счастьем»'},
  {'id': '6153b7d49e8e5ae3eb22ff60', 'name': '«Метод Хитча»'},
  {'id': '6153b7d49e8e5ae3eb22ff61', 'name': '«Терминатор 2»'},
  {'id': '6153b7d49e8e5ae3eb22ff62', 'name': '«Человек-Паук 3»'},
  {'id': '6153b7d49e8e5ae3eb22ff63', 'name': 'А и Б.Стругацкие'},
  {'id': '6153b7d49e8e5ae3eb22ff64', 'name': 'А. Азольский'},
  {'id': '6153b7d49e8e5ae3eb22ff65', 'name': 'А. Бергсон'},
  {'id': '6153b7d49e8e5ae3eb22ff66', 'name': 'А. Геров'},
  {'id': '6153b7d49e8e5ae3eb22ff67', 'name': 'А. Грамши'},
  {'id': '6153b7d49e8e5ae3eb22ff68', 'name': 'А. Граф'},
  {'id': '6153b7d49e8e5ae3eb22ff69', 'name': 'А. Додэ'},
  {'id': '6153b7d49e8e5ae3eb22ff6a', 'name': 'А. Дюма'},
  {'id': '6153b7d49

In [5]:
author_name = 'A.J. Styles'

author_id = None
author_language = None
for lang in authors_list.keys():
    for item in authors_list[lang]:
        if 'name' in item.keys() and item['name'] == author_name:
            author_id = item['id']
            author_language = lang
print(author_id, author_language)

6153b7d49e8e5ae3eb2306a3 en


In [6]:
def get_author_quotes(language_id, author_id):
    response = requests.get(f"{BASE_URL}/authors/{language_id}/{author_id}/quotes")
    quotes = response.json()
    return quotes

# Fetch quotes by a specific author
author_quotes = get_author_quotes(author_language, author_id)
author_quotes


[{'id': '6153bbb29e8e5ae3eb23a4f4',
  'text': "You just remember back when you were watching as a kid and going, 'Man, Sting's so cool,' and now I'm wrestling the guy. It's breathtaking.",
  'author': {'id': '6153b7d49e8e5ae3eb2306a3', 'name': 'A.J. Styles'}},
 {'id': '6153bbb49e8e5ae3eb23f458',
  'text': "At my house, it's an, 'If dad says it, you can say it' kind of deal, so a lot of my slang words come off very childish at this point in my career.",
  'author': {'id': '6153b7d49e8e5ae3eb2306a3', 'name': 'A.J. Styles'}},
 {'id': '6153bbb79e8e5ae3eb24974a',
  'text': "I want to be known as A.J. Styles, the WWE Superstar that he is, and have amazing matches, make memories - I think that's the goal.",
  'author': {'id': '6153b7d49e8e5ae3eb2306a3', 'name': 'A.J. Styles'}},
 {'id': '6153bbb89e8e5ae3eb24c8a9',
  'text': "I've never really felt like a veteran. I've never felt like the guy who's like, 'OK, everyone needs to look up to me and respect me.' I've always just been one of the guys

In [7]:
def like_quote(language_id, quote_id):
    payload = {
        "likeQuote": {
            "quote-id": quote_id
        }
    }
    
    # Send POST request with JSON payload
    response = requests.post(f"{BASE_URL}/likes/{language_id}", json=payload)
    print(response.content)
    
    # Check if the request was successful
    if response.status_code == 200:
        print("Quote liked successfully!")
    else:
        print(f"Failed to like the quote. Status code: {response.status_code}, Response: {response.text}")

# Like a specific quote
quoteId = "6153bbb29e8e5ae3eb23a4f4"
is_liked = like_quote("en", quoteId)
print(f"Quote liked: {is_liked}")


b'{"message":"Unauthorized","_links":{"self":{"href":"/v1/likes/en","templated":false}},"_embedded":{"errors":[{"message":"Unauthorized"}]}}'
Failed to like the quote. Status code: 401, Response: {"message":"Unauthorized","_links":{"self":{"href":"/v1/likes/en","templated":false}},"_embedded":{"errors":[{"message":"Unauthorized"}]}}
Quote liked: None
